In [1]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
from neuralStellar2 import stellarGrid,NNmodel
from datetime import datetime
from matplotlib import rc
rc("font", family="serif", size=14)
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l1, l2
import pickle

config = tf.ConfigProto(device_count = {'GPU': 0})
sess = tf.Session(config=config)
config.gpu_options.allow_growth = True

Using TensorFlow backend.


In [0]:
def set_seed(self, seed):
    ''' Set the seed '''
    np.random.seed(seed)
    tf.random.set_random_seed(seed)
NNmodel.set_seed = set_seed

grid_file='drive/My Drive/4th Year Project/grid_0_0.csv'
df = pd.read_csv(grid_file)

df['tenstarfeh'] = 10**df.star_feh # not required because already in grid
df['scale_T'] = (df.effective_T/5000) #adds additional column for experimenting
df = df.sample(frac=1).reset_index(drop=True) #shuffling
#print(df.keys())
x_cols = ['star_mass', 'star_age', 'tenstarfeh', 'star_MLT'] #inputs cols
y_cols = ['radius', 'scale_T', 'delta_nu'] # output cols

In [40]:
#arch = [len(x_cols),'bn',64,64,64,64,64,64, len(y_cols)]
arch = [len(x_cols),16,16,16,16, len(y_cols)]
#arch = [len(x_cols), 'bn',128,128,128,128,128,128,128,128,256,256, len(y_cols)]
m1=NNmodel('evo',['mass', 'age', 'feh', 'MLT'], ['radius', 'Teff', 'delnu'])
m1.buildModel(arch=arch, activation='elu',reg=['l1',0.00001],dropout=None)
m1.compileModel(opt='Nadam', lr=0.001, loss='MAE', metrics=['MAE','MSE'], beta_1=0.9995, beta_2=0.999) #
m1.fitModel(df=df, cols=[x_cols,y_cols], epoch_no=20, batch_size=1000000, save_name='drive/My Drive/4th Year Project/{}/{}'.format("test",""))

Model: "neuralstellar"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_7 (Dense)              (None, 16)                80        
_________________________________________________________________
dense_8 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_9 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 51        
Total params: 947
Trainable params: 947
Non-trainable params: 0
_______________________________________________________

In [0]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
import keras.backend as K
import numpy as np
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

ipt = Input((12,))
out = Dense(12)(ipt)
model = Model(ipt, out)
model.compile(SGD(1e-4, decay=1e-2, momentum = 0, nesterov = False), loss='mse')

x = y = np.random.randn(32, 12)  # dummy data
for iteration in range(10):
    model.train_on_batch(x, y)
    y_true = model.optimizer._decayed_lr('float32').numpy()
    #print(y_true)
    #y_true = K.print_tensor(y_true, message='y_true = ')
    #print(type(model.optimizer._decayed_lr('float32')))
    #print("lr at iteration {}: {}".format(iteration + 1, model.optimizer._decayed_lr('float32')))


In [9]:
print(model.optimizer._decayed_lr('float32').numpy())


9.090909e-05


{'name': 'model_1', 'layers': [{'name': 'input_2', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 12), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_2'}, 'inbound_nodes': []}, {'name': 'dense_1', 'class_name': 'Dense', 'config': {'name': 'dense_1', 'trainable': True, 'dtype': 'float32', 'units': 12, 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None, 'dtype': 'float32'}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {'dtype': 'float32'}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}, 'inbound_nodes': [[['input_2', 0, 0, {}]]]}], 'input_layers': [['input_2', 0, 0]], 'output_layers': [['dense_1', 0, 0]]}


In [0]:
"""
#dir(m1.model.optimizer)
 '_compute_gradients',
 '_decayed_lr',
 '_gather_saveables_for_checkpoint',
 '_get_hyper',
 '_initial_decay',
 '_iterations',
 '_set_hyper',
 '_track_trackable',
 '_tracking_metadata',
 '_weights',
 'add_slot',
 'add_weight',
 'apply_gradients',
 'epsilon',
 'from_config',
 'get_config',
 'get_gradients',
 'get_slot',
 'get_updates',
 'get_weights',
 'iterations',
 'minimize',
 'set_weights',
 'variables',
 'weights'
"""